# Samurai Gunn map tile classification

#### First we define the window capture function
On it's own, it seems to barely have any performance impact, outputting up to 800-900 fps when capturing the windowed version of Samurai Gunn, a 320x240 px window. 

In [1]:
import cv2 as cv
import numpy as np
from time import time
import win32gui
import win32ui
import win32con

def list_all_open_windows():
    def winEnumHandler( hwnd, ctx ):
        if win32gui.IsWindowVisible( hwnd ):
            print (hex(hwnd), win32gui.GetWindowText( hwnd ))

    win32gui.EnumWindows(winEnumHandler, None)

#list_all_open_windows()

def window_capture():
    hwnd = win32gui.FindWindow(None, 'Samurai Gunn')
    
    # Getting the window's size and accounting for window screenshot borders
    #window_rect = win32gui.GetWindowRect(hwnd)
    titlebar_px = 38
    border_px = 9

    # For samurai Gunn, the non-fullscreen dimensions should be:
    w = 320
    h = 240
    
    crop_x = border_px
    crop_y = titlebar_px
    
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h) , dcObj, (crop_x, crop_y), win32con.SRCCOPY)
    
    # saving screenshot to file
    # To save screenshot to file, uncomment this 
#     bmpfilenamename = "sample" + str(img_counter) + ".jpg" #set this
#     dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)
    
    # Converting to format useful for opencv
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    img = np.frombuffer(signedIntsArray, dtype='uint8')
    img.shape = (h, w, 4)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())
    
    
    # Dropping alpha channel may be useful for some applications, like cv.matchTemplate()
    # which may throw an error otherwise
    #img = img[...,:3]   # this drops alpha channel 
    
    return img

## Defining Tile classification functions
We can notice that each map tile sits in a grid of 15x20 tiles, with each tile being (16x16)px, with some exceptions that are offset vertically by half a tile.
Taking advantage of this, we can simplify the input given to our neural network by reducing each tile to a single pixel that represents it.

It's important to note that even if the map tiles can only ever be on this discrete grid, the players are free to move in the full continuous plane. This must be taken into consideration when classifying tiles that contain players.


In [40]:

row_step = 2
col_step = 2
def classify_tile(x, y, img, sliced):
    '''
    Tiles are (16 x 16)px
    Parameters:
    x, y  : coords of the upper left corner of the tile
    img: input image
    sliced: int, either 0 or 1, letting us know if it's a full tile or a half tile at the 
             upper/lower edges of the map. 
             0 means it's a full tile
             1 means it's a half tile
    Returns:
    int: Returns a single int that will classify the tile
    '''
    px_count = 0
    total = np.zeros((1, img.shape[2]), dtype='uint32')
    
    corrected_height_range = int(16/(1+sliced))
    for i in range(0, corrected_height_range, row_step):
        for j in range(0, 16, col_step):
            total += img[x+i][y+j] # [r, g, b, a]
            px_count += 1           
    
    #result = result.astype('uint8') 
    # for some reason this doesn't work. It's fine tho, it works if
    # we convert the data type before returning on the simplify() funct
    result = total * ((1+sliced)/(px_count))
    
    return result
    

In [39]:

def simplify(img, offset):
    '''Reduces the input image resolution by classifying the tiles on the screen and 
    reducing them to 1 px per tile.
    The tiles in every map can be aligned with a 20 x 15 grid of (16 x 16)px cells.
    Iterates over each tile, calling classify_tile() for each of them
    
    Parameters:
    img: input image
    offset: in case the map tiles do not align perfectly with the grid, an offset of 8px down does 
            the trick. (Maps with different offsets have yet to be found, 
            this should allow for different offsets in the X direction)
            
    Returns:
    numpy 3D array: (15 x 20 x num_channels) numpy array
    '''
    sliced = 0
    simple_img = np.zeros((15, 20, img.shape[2]))
    for x in range(15):
        if offset == 8 and (x == 0 or x == 19):
            sliced = 1
        else:
            sliced = 0
        for y in range(20):
            # pass the coordinates of each tile with corrections to accomodate for the grid offset
            simple_img[x, y] = classify_tile(x*16 - (offset*(1-sliced)), 
                                             y*16 - (offset*(1-sliced)), 
                                             img, sliced)
    simple_img = simple_img.astype('uint8')
    return simple_img

## Main Loop

In [37]:
frame_count = 1
cumulative_fps = 0

# offset will depend on the map
# for Ice cube, offset is 0
offset = 0

while(True):
    prev_time = time()
    
    screenshot = window_capture()
    #screenshot = cv.imread('sample0.jpg')
    img = simplify(screenshot, offset)
    dim = (img.shape[1] * 16, img.shape[0] * 16)
    resized = cv.resize(img, dim, interpolation = cv.INTER_AREA) #  interpolation = cv.INTER_AREA

    cv.imshow('Simplified', resized)
    cv.imshow('Screenshot', screenshot)
    #print('FPS {}'.format(1 / (time() - prev_time)))
     
    cumulative_fps += 1 / (time() - prev_time)
    print('Avg FPS {}'.format(cumulative_fps/frame_count))
    frame_count += 1
    
    if frame_count > 1000:
        frame_count = 0
        cumulative_fps = 0
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

Avg FPS 8.644662915558854
Avg FPS 10.833123003367188
Avg FPS 12.361461326870975
Avg FPS 13.056104333479935
Avg FPS 13.528363709386412
Avg FPS 13.971059459609227
Avg FPS 14.248860823293581
Avg FPS 14.160692311282167
Avg FPS 14.413461844821548
Avg FPS 14.539476265414823
Avg FPS 14.663871307375041
Avg FPS 14.688872999609805
Avg FPS 14.802978512819411
Avg FPS 14.733362073857862
Avg FPS 14.811562858583853
Avg FPS 14.865477165942178
Avg FPS 14.884232631811868
Avg FPS 14.986391932724757
Avg FPS 15.048680540322446
Avg FPS 15.124214565610213
Avg FPS 15.188460584464265
Avg FPS 15.244856932701142
Avg FPS 15.25699586582309
Avg FPS 15.254322926362823
Avg FPS 15.234059961031743
Avg FPS 15.25938965858651
Avg FPS 15.313737007065097
Avg FPS 15.353877803114646
Avg FPS 15.393480389613122
Avg FPS 15.402566180419862
Avg FPS 15.36880254774037
Avg FPS 15.389695580751717
Avg FPS 15.353614367762574
Avg FPS 15.356836207668136
Avg FPS 15.392298708470442
Avg FPS 15.368385269106504
Avg FPS 15.340144800474794
Avg F